**Imports**

In [6]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
from PIL import Image

from keras.models import Model
from keras.models import load_model

**Paths**

In [7]:
test_csv = '/kaggle/input/UBC-OCEAN/test.csv'
test_images = '/kaggle/input/UBC-OCEAN/test_images/'
test_thumbnails = '/kaggle/input/UBC-OCEAN/test_thumbnails/'
model_file = '/kaggle/input/ml-course-project-model/model.h5'

classes = ['CC', 'EC', 'HGSC', 'LGSC', 'MC']

**Functions**

In [8]:
def one_hot_to_values(vector, classes):
    values = []
    for one_hot in vector:
        value = classes[np.argmax(one_hot)]
        values.append(value)
    return np.array(values)

def load_image(idx):
    try:
        image = Image.open(test_thumbnails+str(idx)+'_thumbnail.png')
    except:
        image = Image.open(test_images+str(idx)+'.png')
    image = image.resize((224,224)) 
    image = np.array(image)
    return image

**Loading model**

In [9]:
model = load_model(model_file)

**Loading test images and making predictions**

In [10]:
test_df = pd.read_csv(test_csv)

test_image_ids = []
test_images = []
for idx in test_df['image_id']:
    image = load_image(idx)
    test_image_ids.append(idx)
    test_images.append(image)

test_images = np.array(test_images).reshape(-1, 224, 224, 3)

test_predicted = one_hot_to_values(model.predict(test_images), classes)

test_df["label"] = test_predicted

submission_df = test_df[["image_id", "label"]]
submission_df.to_csv("submission.csv", index=False)

1/1 [==============================] - 1s 977ms/step
